<a href="https://colab.research.google.com/github/galexbh/afp-optimizer/blob/main/src/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizador de aporte AFP (Honduras)

Incluye explicaciones, glosario y notas directamente en la UI.

Ajusta tramos ISR y límites legales si cambian.

In [ ]:
# %%capture
!pip -q install gradio pandas numpy

# %%
import gradio as gr
import pandas as pd
import numpy as np
import tempfile

# ========== Parámetros legales/tributarios (ajústalos si cambian) ==========
TRAMOS_ISR = [
    (0,       223_400,     0.00),  # Exento
    (223_400, 341_000,     0.15),
    (341_000, 591_000,     0.20),
    (591_000, float("inf"),0.25),
]
UMBRAL_EXENTO_MENSUAL = 21_457.76  # Referencia útil para explicaciones en la UI

# ========== Funciones financieras ==========
def calcular_isr_anual(salario_mensual, tramos=TRAMOS_ISR):
    ingreso_anual = salario_mensual * 12
    impuesto = 0.0
    for li, ls, t in tramos:
        if ingreso_anual > li:
            base = min(ingreso_anual, ls) - li
            impuesto += base * t
        else:
            break
    return impuesto

def fv_aportes_constantes(aporte_anual, tasa_anual, n_anios):
    """Futuro de una anualidad (aportes al fin de cada año) con capitalización anual."""
    if tasa_anual == 0:
        return aporte_anual * n_anios
    return aporte_anual * (((1 + tasa_anual) ** n_anios - 1) / tasa_anual)

def KPIs(salario_mensual, aporte_mensual, tasa_afp, tasa_libre, edad, edad_jub, tope_deducible_anual):
    """KPIs y alertas para un aporte puntual."""
    años = max(0, int(edad_jub - edad))
    r_afp, r_libre = tasa_afp/100, tasa_libre/100
    isr_base = calcular_isr_anual(salario_mensual)
    ingreso_gravable_m = salario_mensual - aporte_mensual
    isr_nuevo = calcular_isr_anual(ingreso_gravable_m)
    ahorro_fiscal_anual = max(isr_base - isr_nuevo, 0.0)
    aporte_anual = aporte_mensual * 12

    cap_afp = fv_aportes_constantes(aporte_anual, r_afp, años)
    cap_libre = fv_aportes_constantes(ahorro_fiscal_anual, r_libre, años)
    cap_total = cap_afp + cap_libre

    tax_alpha = 0.0 if aporte_anual == 0 else ahorro_fiscal_anual / aporte_anual
    rend_comb_instant = tax_alpha + r_afp
    reduccion_isr_pct = 0.0 if isr_base == 0 else (1 - isr_nuevo / isr_base) * 100

    alertas = []
    if aporte_anual > tope_deducible_anual:
        alertas.append(
            f"🚨 Aporte anual (L {aporte_anual:,.0f}) excede tope deducible (L {tope_deducible_anual:,.0f}). "
            "El excedente no deduce ISR."
        )
    if ingreso_gravable_m <= UMBRAL_EXENTO_MENSUAL:
        alertas.append("✅ Con este aporte quedas ~exento de ISR (salario gravable mensual ≤ umbral exento ref.).")

    return {
        "isr_base": isr_base,
        "isr_nuevo": isr_nuevo,
        "ahorro_fiscal_anual": ahorro_fiscal_anual,
        "aporte_anual": aporte_anual,
        "cap_afp": cap_afp,
        "cap_libre": cap_libre,
        "cap_total": cap_total,
        "tax_alpha": tax_alpha,
        "rend_comb_instant": rend_comb_instant,
        "reduccion_isr_pct": reduccion_isr_pct,
        "alertas": alertas
    }

def optimizar_aporte(
    salario_mensual=26_000,
    meta_reduccion_isr_pct=50,
    tope_afp_pct=20,
    tasa_afp=6.0,
    tasa_inversion_libre=7.0,
    edad_actual=26,
    edad_jubilacion=65,
    tope_deducible_anual=100_000.0,
):
    meta_reduccion_isr = meta_reduccion_isr_pct / 100.0
    tope_afp = tope_afp_pct / 100.0
    r_afp = tasa_afp / 100.0
    r_libre = tasa_inversion_libre / 100.0
    anios = max(0, int(edad_jubilacion - edad_actual))

    isr_base = calcular_isr_anual(salario_mensual)

    aporte_max = int(salario_mensual * tope_afp)
    candidatos = []
    alertas_full_map = {}  # clave: aporte_m -> texto completo de alertas
    for aporte_m in range(0, aporte_max + 100, 100):  # barrido cada L100
        ingreso_grav_m = salario_mensual - aporte_m
        isr = calcular_isr_anual(ingreso_grav_m)
        # Condición: alcanzar meta de reducción de ISR
        if isr <= isr_base * (1 - meta_reduccion_isr):
            aporte_anual = aporte_m * 12
            capital_afp = fv_aportes_constantes(aporte_anual, r_afp, anios)
            ahorro_fiscal_anual = max(isr_base - isr, 0)
            capital_libre = fv_aportes_constantes(ahorro_fiscal_anual, r_libre, anios)
            total = capital_afp + capital_libre

            k = KPIs(salario_mensual, aporte_m, tasa_afp, tasa_inversion_libre,
                     edad_actual, edad_jubilacion, tope_deducible_anual)

            # Resumen corto de alertas (para no romper la tabla)
            if not k["alertas"]:
                alert_short = ""
            else:
                # Muestra solo el primer aviso de forma corta
                prim = k["alertas"][0]
                alert_short = prim if len(prim) <= 60 else prim[:57] + "…"

            # Guardar alertas completas para el panel de detalle
            alertas_full_map[aporte_m] = "\n".join(k["alertas"]) if k["alertas"] else "—"

            candidatos.append({
                "Aporte mensual (L)": aporte_m,
                "ISR anual (L)": round(isr, 2),
                "Reducción ISR (%)": round(k["reduccion_isr_pct"], 2),
                "Ahorro fiscal anual (L)": round(ahorro_fiscal_anual, 2),
                "Tax Alpha (%)": round(k["tax_alpha"] * 100, 2),
                "Rend. combinado instantáneo (%)": round(k["rend_comb_instant"] * 100, 2),
                "Capital AFP a retiro (L)": round(capital_afp, 2),
                "Capital libre a retiro (L)": round(capital_libre, 2),
                "Capital total a retiro (L)": round(total, 2),
                "🔔 Alertas": alert_short,   # <- columna compacta
            })

    if not candidatos:
        msg = (
            "No encontré un aporte que cumpla la meta de reducción de ISR con el tope de AFP actual.\n\n"
            "💡 Sugerencias:\n"
            "• Aumenta el tope de aporte a AFP (p. ej., 25–30%).\n"
            "• Reduce la meta de reducción de ISR (p. ej., 30–40%).\n"
            "• Verifica/actualiza los tramos ISR vigentes o el tope deducible.\n"
        )
        return pd.DataFrame(), {}, msg, [], "", {}

    df = pd.DataFrame(candidatos)
    fila_optima = df.iloc[df["Capital total a retiro (L)"].idxmax()].to_dict()

    # KPIs de la opción óptima + beneficio marginal por +L100
    aporte_m_opt = int(fila_optima["Aporte mensual (L)"])
    k_opt = KPIs(salario_mensual, aporte_m_opt, tasa_afp, tasa_inversion_libre,
                 edad_actual, edad_jubilacion, tope_deducible_anual)
    k_opt_plus = KPIs(salario_mensual, aporte_m_opt + 100, tasa_afp, tasa_inversion_libre,
                      edad_actual, edad_jubilacion, tope_deducible_anual)
    beneficio_marginal = round(k_opt_plus["cap_total"] - k_opt["cap_total"], 2)

    resumen = {
        "💼 Salario mensual (L)": salario_mensual,
        "🧾 ISR base anual (L)": round(isr_base, 2),
        "🎯 Meta reducción ISR (%)": meta_reduccion_isr_pct,
        "🧱 Tope AFP (% salario)": tope_afp_pct,
        "📈 Tasa AFP nominal anual (%)": tasa_afp,
        "💹 Tasa inversión libre anual (%)": tasa_inversion_libre,
        "⏳ Años hasta retiro": anios,
        "🔒 Tope deducible anual (L)": tope_deducible_anual,
        "💡 Aporte óptimo (L/mes)": aporte_m_opt,
        "🧮 ISR anual con óptimo (L)": round(k_opt["isr_nuevo"], 2),
        "🟢 Reducción ISR con óptimo (%)": round(k_opt["reduccion_isr_pct"], 2),
        "⚡ Tax Alpha con óptimo (%)": round(k_opt["tax_alpha"] * 100, 2),
        "➕ Rend. combinado instantáneo (%)": round(k_opt["rend_comb_instant"] * 100, 2),
        "➕ Beneficio marginal por +L100": beneficio_marginal,
        "🏦 Capital AFP a retiro (L)": round(k_opt["cap_afp"], 2),
        "📦 Capital libre a retiro (L)": round(k_opt["cap_libre"], 2),
        "💰 Capital total a retiro (L)": round(k_opt["cap_total"], 2),
    }

    nota = (
        f"Tip: para minimizar ISR, puedes aportar lo suficiente para que el salario gravable mensual "
        f"quede ≤ L {UMBRAL_EXENTO_MENSUAL:,.2f}. Revisa también el tope deducible anual."
    )

    # Lista de aportes disponibles para el selector
    aportes_disponibles = [int(x) for x in df["Aporte mensual (L)"].tolist()]
    # Alertas completas del aporte óptimo (valor por defecto del panel)
    alertas_opt = alertas_full_map.get(aporte_m_opt, "—")

    return df, resumen, nota, aportes_disponibles, alertas_opt, alertas_full_map

def exportar_csv(df_tabla, resumen):
    """CSV: opciones y resumen."""
    files = []
    if df_tabla is not None and not df_tabla.empty:
        f1 = tempfile.NamedTemporaryFile(delete=False, suffix="_opciones.csv")
        df_tabla.to_csv(f1.name, index=False, encoding="utf-8-sig")
        files.append(f1.name)
    if resumen:
        f2 = tempfile.NamedTemporaryFile(delete=False, suffix="_resumen.csv")
        pd.DataFrame([resumen]).to_csv(f2.name, index=False, encoding="utf-8-sig")
        files.append(f2.name)
    return files if files else None

# ========== UI ==========
with gr.Blocks(title="Optimizador AFP Honduras — tabla limpia") as demo:
    gr.Markdown(
        """
        # 🧮 Optimizador de aporte AFP (Honduras)
        Balancea **menos ISR hoy** y **más capital al retiro**.
        """
    )

    with gr.Accordion("🧠 ¿Cómo funciona el cálculo?", open=False):
        gr.Markdown(
            f"""
            1. Calculamos tu **ISR anual base** con tu salario mensual actual (sin AFP).
            2. Probamos aportes mensuales a AFP (en saltos de L 100) hasta tu **tope de AFP**.
            3. Filtramos los aportes que logran la **meta de reducción de ISR** (ej. bajar 50% vs. base).
            4. Para cada aporte válido:
               - Proyectamos **capital AFP** (tasa AFP) y **capital 'libre'** invirtiendo el **ahorro fiscal**.
               - Sumamos ambos y elegimos el que **maximiza el capital total**.

            > Umbral exento mensual de referencia: **L {UMBRAL_EXENTO_MENSUAL:,.2f}**.
            """
        )

    with gr.Accordion("📘 Glosario rápido", open=False):
        gr.Markdown(
            """
            **💼 Salario mensual**
            Dinero que recibes cada mes **antes** de descuentos o deducciones.

            **💰 AFP (Administradora de Fondos de Pensiones)**
            Empresa que administra tus ahorros para el retiro.
            • **Ventaja:** lo que aportas a la AFP puede **deducir ISR** (hasta un límite legal).
            • **Desventaja:** no es dinero líquido; es para tu **jubilación** salvo excepciones.

            **🧾 ISR (Impuesto Sobre la Renta)**
            Impuesto que se paga sobre tu ingreso **anual**. En Honduras es **progresivo por tramos**:
            • Primer tramo: **exento** (no paga).
            • Siguientes tramos: pagan distintos **porcentajes**.

            **📌 Umbral exento**
            Ingreso **mensual** máximo que puedes tener **sin pagar ISR**.
            Si tu salario **después de aportar a AFP** queda por debajo de este umbral, no pagas ISR.

            **🎯 Meta de reducción de ISR**
            Cuánto quieres **bajar** tu impuesto comparado con no aportar a AFP.
            Ej.: si hoy pagarías L 10,000 y fijas meta **50%**, apuntas a pagar **L 5,000**.

            **🧱 Tope AFP**
            Máximo **% de tu salario** que estás dispuesto a aportar a la AFP (preferencia personal).
            Ej.: Tope 20% con salario L 20,000 → aporte máx. **L 4,000/mes**.

            **📈 Rendimiento AFP (nominal anual)**
            Porcentaje al que crecen tus aportes **dentro** de la AFP cada año (ej.: 6% anual).

            **💹 Rendimiento inversión libre**
            Porcentaje al que crecería el **ahorro fiscal** si lo inviertes por tu cuenta (plazos, fondos, etc.).

            **💸 Ahorro fiscal anual**
            Lo que **no pagas** de ISR gracias a tus aportes a la AFP.
            Ej.: ISR antes L 8,000 → ahora L 5,000 → **ahorro L 3,000**.

            **🏦 Capital AFP a retiro**
            Suma de **todos** tus aportes en AFP + intereses, hasta la edad de jubilación.

            **📊 Capital libre a retiro**
            Suma de **todo** lo que invertiste por tu cuenta usando el **ahorro fiscal**, más sus rendimientos.

            **🔀 Capital total a retiro**
            **AFP + Libre**. Es la cifra que más importa para tu retiro.

            **⚖️ Tax Alpha**
            “Rendimiento inmediato” por beneficio fiscal:
            Sirve para comparar contra otras inversiones: cuanto más alto, mejor.

            **➕ Rendimiento combinado instantáneo**
            Aproximación rápida: **Tax Alpha + rendimiento AFP**. Útil para comparar alternativas.

            **📅 Horizonte de inversión**
            Años desde tu edad actual hasta tu jubilación.
            Más años = mayor efecto del **interés compuesto**.

            **⚠️ Tope deducible anual (legal)**
            Límite máximo de aportes que **pueden deducir ISR** en el año.
            Si lo excedes, el **excedente** ya **no** reduce impuestos (aunque se invierte en AFP).
            """
        )

    with gr.Row():
        salario = gr.Number(value=26_000, label="💼 Salario mensual (L)", precision=0)
        edad = gr.Slider(value=26, minimum=18, maximum=60, step=1, label="🎂 Edad actual")
        edad_jub = gr.Slider(value=65, minimum=55, maximum=70, step=1, label="🎯 Edad de jubilación")

    with gr.Row():
        meta_isr = gr.Slider(value=50, minimum=0, maximum=90, step=5, label="🎯 Meta de reducción de ISR (%)")
        tope_afp = gr.Slider(value=20, minimum=5, maximum=50, step=1, label="🧱 Tope aporte AFP (% del salario)")

    with gr.Row():
        tasa_afp = gr.Slider(value=6.0, minimum=0.0, maximum=15.0, step=0.1, label="📈 Rendimiento AFP anual (%)")
        tasa_libre = gr.Slider(value=7.0, minimum=0.0, maximum=20.0, step=0.1, label="💹 Rendimiento inversión libre (%)")

    tope_deducible = gr.Number(value=100_000, label="🔒 Tope deducible anual (L)", precision=0)

    ejecutar = gr.Button("✅ Calcular mejor opción")

    gr.Markdown("### 📋 Opciones que cumplen la meta (alertas resumidas)")
    tabla = gr.Dataframe(
        label="",
        wrap=False,
        interactive=False,
    )

    gr.Markdown("### 🏆 Resumen de la opción óptima")
    resumen = gr.JSON(label="", show_label=False)

    gr.Markdown("### 📊 KPIs clave (óptimo)")
    kpis = gr.JSON(label="", show_label=False)

    gr.Markdown("### 🔎 Alertas completas de la fila seleccionada")
    with gr.Row():
        selector_aporte = gr.Dropdown(choices=[], label="Elige aporte (L/mes)")
        alerta_detalle = gr.Textbox(label="Detalle de alertas", lines=5)

    gr.Markdown("### 📝 Notas")
    nota = gr.Textbox(label="", interactive=False)

    archivos = gr.Files(label="⬇️ Descargas CSV")

    # Estado interno para mapear aporte -> alertas completas
    alerts_state = gr.State({})

    def wrapper_optimizar(sal, meta, tope, r_afp, r_libre, e, ej, tope_ded):
        df, res, nt, aportes_list, alerta_opt, alert_map = optimizar_aporte(
            sal, meta, tope, r_afp, r_libre, e, ej, tope_ded
        )
        # KPIs clave desde el resumen (reconstruimos de forma breve)
        kpi_block = {
            "⚡ Tax Alpha (%)": res.get("⚡ Tax Alpha con óptimo (%)", None),
            "➕ Rend. combinado instantáneo (%)": res.get("➕ Rend. combinado instantáneo (%)", None),
            "🟢 Reducción ISR con óptimo (%)": res.get("🟢 Reducción ISR con óptimo (%)", None),
            "➕ Beneficio marginal por +L100": res.get("➕ Beneficio marginal por +L100", None),
        }
        files = exportar_csv(df, res)
        return df, res, kpi_block, nt, gr.update(choices=aportes_list, value=aportes_list[0] if aportes_list else None), alerta_opt, files, alert_map

    ejecutar.click(
        fn=wrapper_optimizar,
        inputs=[salario, meta_isr, tope_afp, tasa_afp, tasa_libre, edad, edad_jub, tope_deducible],
        outputs=[tabla, resumen, kpis, nota, selector_aporte, alerta_detalle, archivos, alerts_state]
    )

    def mostrar_alerta_detalle(aporte, alert_map):
        if not aporte:
            return "—"
        return alert_map.get(int(aporte), "—")

    selector_aporte.change(
        fn=mostrar_alerta_detalle,
        inputs=[selector_aporte, alerts_state],
        outputs=alerta_detalle
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e9cb3a0885f5d073d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
